In [2]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import random
%matplotlib inline

In [3]:
G=nx.barabasi_albert_graph(3000,10)

In [5]:
def RD(G,S): #Input the network, times of simulation; return the average density r

    #The parameters
    a=0.4; b=0.2; p=0.2 # alpha, lambda and p 
    
    N=G.number_of_nodes()
    ranks=nx.degree_centrality(G)
    nodes=sorted(G.nodes(), key=lambda n: ranks[n], reverse=True)
    
    for ipt_p in np.linspace(0,2,num=11,endpoint=True):    #different in different model
        num_error_nodes=N*ipt_p*0.01
        M=int(num_error_nodes)
        Gx=G.copy()
        for i in range(M):
            Gx.remove_node(nodes[i])
            
        final_statistics=[]
        for seed in Gx.nodes(): #iteratively set seed of anti-rumor
            statistics=[]
            for i in Gx.nodes():
                Gx.node[i]['rumor']=0 #Ignorant=0, Spreader=1 and Stifler=2   ???
            Gx.node[seed]['rumor']=1
    
            for simulation in range(0,S):
                G1=Gx.copy()
                statistics1=[0,1,0]
                while statistics1[1]>0: #Stop if no spreaders of R and AR
                    G2=G1.copy()
                    for i in G1.nodes():
                        if G1.node[i]['rumor']==1: # if 1, then spreads  
                            for j in G1.neighbors(i):
                                if G1.node[j]['rumor']==0:
                                    r1=random.random()
                                    if r1<a: #if satisfied, accept the rumor
                                        r2=random.random()
                                        if r2<p:
                                            G2.node[j]['rumor']=1
                                        else:
                                            G2.node[j]['rumor']=2
                                else:
                                    r3=random.random()
                                    if r3<b: #if satisfied, S stops to spread
                                        G2.node[i]['rumor']=2        
                
                    n_s=0;n_i=0;n_r=0 #Calculate number of S,I,R or rumor
                    for k in G2.nodes():
                        if G2.node[k]['rumor']==1:
                            n_s=n_s+1
                        elif G2.node[k]['rumor']==0:
                            n_i=n_i+1
                        else:
                            n_r=n_r+1
                    statistics1=[n_i,n_s,n_r]
                    G1=G2.copy()
                statistics.append(statistics1) #The results for each simulation are stored in it.
            final_statistics.append(statistics)
        
        #The average density of stifler
        N1=float(Gx.number_of_nodes());S1=float(S);density_r=0
        for i in final_statistics:
            for j in i:
                    density_r=density_r+j[2]/N1
        print(density_r/(S1*N1))
        

In [ ]:
time RD(G,10)